In [122]:
import numpy as np
import copy
import sklearn.tree as sc
import time

In [112]:
# Load full data
path_real_data = "./zipcombo.dat"
data = np.loadtxt(path_real_data)
data_samples = data[:, 1:]; data_labels = data[:, 0].astype(int)

In [237]:
def gini_index(class_counts, total_count):
    """
    Input: length num_class of class counts, and total samples (scalar)
    Output: gini index 
    """
    props = class_counts / total_count
    gini = np.sum(props * (1-props))
    return gini

def rel_entropy(class_counts, total_count):
    """
    Input: length num_class of class counts, and total samples (scalar)
    Output: relative entropy (log base 2)
    """
    props = class_counts / total_count
    # Eliminate zero proportions. x log (1/x) = 0 at x = 0 or 1
    props += 1 * (props == 0)
    entropy = np.sum(props * np.log2(1/props))
    return entropy

def best_split_along_feature(feature_vect, labels, num_classes, min_samples, split_function):
    """
    Input: Length m vectors of feature values and labels
    Output: gini/entropy value of the split, and feature value for the split
    """
    # Initially, all belong to left. Initialise counts and total samples in left and right portions. As we move
    # through splits, just change these counts.
    
    n_points = labels.shape[0]
    cc_left = np.zeros((num_classes))
    
    # Left counts: Initially, everything
    left_labels, left_counts = np.unique(labels, return_counts = True)
    cc_left[left_labels] += left_counts
    tot_count_left = n_points
    
    # Right counts: Initially nothing
    cc_right = np.zeros((num_classes))
    tot_count_right = 0
    
    # Initially, entropy/gini of the whole group, i.e. left, and split value is +inf, means everything is one region
    # Also return the counts of the best split, so we don't have to re-count later
#     best_val = split_function(cc_left, tot_count_left)
    best_val = np.inf # I.e. we can't allow all - empty split
    split_val = -np.inf 
    cc_right_final = np.zeros((num_classes)); cc_left_final = copy.deepcopy(cc_left)
    
    # Sort the feature vect, so we can find split points in one-pass left to right
    sorted_idx = np.argsort(feature_vect)
    sorted_labels = labels[sorted_idx]; sorted_fv = feature_vect[sorted_idx]
    
    # iterate through the sorted feature vector, and compute the quality of each split.
    start_idx = 0; end_idx = 1
    # New addition to the right is always the chunk between start and end idx. This is to move chunks of labels with same feature values
    # Follow indexing convention, the end is excluded
    while start_idx < n_points:
        while end_idx < n_points and (sorted_fv[end_idx] == sorted_fv[end_idx - 1]):
            end_idx += 1

        chunk = sorted_labels[start_idx : end_idx]
        chunk_labels, counts = np.unique(chunk, return_counts = True)
        
        print(cc_left, cc_right)
        # Move counts to the right portion
        cc_left[chunk_labels] -= counts
        cc_right[chunk_labels] += counts

        tot_count_left -= np.sum(counts)
        tot_count_right += np.sum(counts)
        
        # Since tot_count_left always decreases, once it decrease below minimum, we break
#         if tot_count_left < min_samples:
#             break
        
        # Re-compute the gini/entropy
        if tot_count_left != 0:
            new_val = split_function(cc_left, tot_count_left) + split_function(cc_right, tot_count_right)
        else:
            # I.e. at the end, where everything is to the right (technically same as starting situation)
            new_val = split_function(cc_right, tot_count_right)
        
        # If new split has less entropy/gini, while still obeying minimum samples requirement
        # we choose this to be the best. We want to find the one with minimum gini
        if new_val < best_val: #and tot_count_left >= min_samples and tot_count_right >= min_samples:
            best_val = new_val
            split_val = sorted_fv[start_idx] # by convention, <= the boundary goes to the right bcs of how we added counts above
            cc_right_final = copy.deepcopy(cc_right); cc_left_final = copy.deepcopy(cc_left)

        start_idx = end_idx; end_idx += 1
        
    print("Results are ", cc_left_final, cc_right_final, split_val)
    return best_val, split_val, cc_left_final, cc_right_final

def find_split_rf(data, labels, n_features, num_classes, min_samples, split_arg = 'gini'):
    """
    Input: m x n of data and vector of length m, labels. n_featues = how many features to consider out of all data features
    Output: dictionary as {feature_idx, feature_value, left and right groups}
    """
    
    # Randomly choose a subset of n_features
    m,n = data.shape
    
    # Initialise
    feature_choices = np.random.choice(np.arange(n), replace = False, size = (n_features,))
    best_val = np.inf; split_val = 0; feature_sidx = 0
    split_function = gini_index if split_arg == 'gini' else rel_entropy
    
    cc_left_final = 0; cc_right_final = 0
    
    # Iterate through features to find the best feature and corresponding value to split on
    for f_idx in feature_choices:
        feature_vect = data[:, f_idx]
        split_qual, split_bound, cc_left, cc_right = best_split_along_feature(feature_vect, labels, num_classes, min_samples, split_function)
        
        if split_qual <= best_val:
            feature_sidx = f_idx
            best_val = split_qual
            split_val = split_bound
            
            cc_left_final = copy.deepcopy(cc_left); cc_right_final = copy.deepcopy(cc_right)
            
            
            
    print("Results", feature_sidx, split_val)
    # Perform the split, i.e. get the dictionary to return
    # NOTE: <= split_val means goes to the RIGHT. This needs to sync so it matches with the cc variable
    right_idx = data[:, feature_sidx] <= split_val; left_idx = np.logical_not(right_idx)
    
    left_group = (data[left_idx,:], labels[left_idx], cc_left_final) # tuple of points, labels, and counts
    right_group = (data[right_idx,:], labels[right_idx], cc_right_final) # tuple of points, labels, and counts
    
    # Create return dictionary to 'characterise' the node, i.e. the split feature, split value of this feature
    # and the left and right groups of this feature for further splitting
    return_dict = {'feature_idx' : feature_sidx,
                   'feature_value' : split_val,
                   'groups' : (left_group, right_group)}
    
    return return_dict

def terminate_node(node):
    """
    Returns the prediction given this 'region' is final/termination
    """
    return np.argmax(node[2])

def isPure(node):
    """
    Returns True if a group contains only data of one label
    """
    label_counts = node[2]
    present_labels = (label_counts != 0)
    
    return (True if np.sum(present_labels) == 1 else False)
    
def rec_split(node, max_depth, min_size, n_features, num_classes, depth, split_arg = 'gini'):
    """
    Recursively split nodes.
    Input:
    node = a dictionary of left and right groups of nodes. Will be recursively passed down
    max_depth = max tree height
    min_size = minimum number of samples in a group
    """
    print('Depth is ', depth)
    
    left_group, right_group = node['groups']
    
    # Delete the groups keys, and replace it with left and right keys to facilitate recursive traversing
    del(node['groups'])
    # check if each group already meet the stopping criterion as follows
    
    # 1.Check if there's an empty group
    if left_group[1].size == 0:
        node['left'] = node['right'] = terminate_node(right_group)
        return
    if right_group[1].size == 0:
        node['left'] = node['right'] = terminate_node(left_group)
        return
    
    # 2. Check if max depth is reached
    if depth >= max_depth:
        node['left'] = terminate_node(left_group)
        node['right'] = terminate_node(right_group)
        return
    
    # 3.1 Process left group
    # If pure, then just terminate that node
    if isPure(left_group):
        node['left'] = terminate_node(left_group)
        
    # If min_size reached, also terminate
    elif left_group[1].size <= min_size:
        node['left'] = terminate_node(left_group)
    
    else:
        # Further split the left. First get the dictionary, and then pass the node to the recursive function
        node['left'] = find_split_rf(left_group[0], left_group[1], n_features, num_classes, min_size, split_arg)
        
        # Now node['left'] contains yet another node dictionary of left and right. Pass this to this recursive function
        # to further split or terminate
        rec_split(node['left'], max_depth, min_size, n_features, num_classes, depth + 1, split_arg)
        
        
    # 3.2 Process right group
    if isPure(right_group):
        node['right'] = terminate_node(right_group)
        
    # If min_size reached, also terminate
    elif right_group[1].size <= min_size:
        node['right'] = terminate_node(right_group)
    
    else:
        # Further split the left. First get the dictionary, and then pass the node to the recursive function
        node['right'] = find_split_rf(right_group[0], right_group[1], n_features, num_classes, min_size, split_arg)
        
        # Now node['left'] contains yet another node dictionary of left and right. Pass this to this recursive function
        # to further split or terminate
        rec_split(node['right'], max_depth, min_size, n_features, num_classes, depth + 1, split_arg)
        
def build_tree(data, labels, max_depth, min_size, n_features, num_classes, split_arg = 'gini'):
    """
    Outer function to build the tree. Do the first split before using the above recursive tree
    """
    root_node = find_split_rf(data, labels, n_features, num_classes, min_size, split_arg)
    
    # Start the recursive splitting
    rec_split(root_node, max_depth, min_size, n_features, num_classes, 1, split_arg)
    
    return root_node

def predict_node(point, node):
    """
    Predict a single data point by recursively traversing the tree
    
    Input:
    point = a length n vector
    """
    feature_idx = node['feature_idx']; boundary_val = node['feature_value']
    
    if point[feature_idx] <= boundary_val:
        # Go to the right
        if isinstance(node['right'], dict):
            prediction = predict_node(point, node['right'])
        else:
            prediction = node['right']
            
    else:
        # Go to the left
        if isinstance(node['left'], dict):
            prediction = predict_node(point, node['left'])
        else:
            prediction = node['left']
    
    return prediction

def predict_tree(data, root_node):
    """
    Predict every point in the data matrix using the fitted tree
    """
    m = data.shape[0]
    preds = np.zeros((m))
    
    for i in range(m):
        pass
        # trave

In [236]:
np.sort(toy_data[:, 1])

array([-1.  , -1.  ,  0.5 ,  0.5 ,  0.8 ,  0.88,  0.88,  0.88])

In [238]:
np.random.seed(88)
toy_data = np.random.choice([-1, 1, 0.5, 0.8, 0.88], size = (8, 3), replace = True)
labels = np.array([0, 0, 2, 1, 2, 1, 0, 1])
result_dict = find_split_rf(toy_data, labels, 3, 5, 1, split_arg = 'gini')
result_dict

[3. 3. 2. 0. 0.] [0. 0. 0. 0. 0.]
[2. 2. 2. 0. 0.] [1. 1. 0. 0. 0.]
[2. 2. 0. 0. 0.] [1. 1. 2. 0. 0.]
[2. 1. 0. 0. 0.] [1. 2. 2. 0. 0.]
Results are  [0. 0. 0. 0. 0.] [3. 3. 2. 0. 0.] 1.0
[3. 3. 2. 0. 0.] [0. 0. 0. 0. 0.]
[1. 3. 2. 0. 0.] [2. 0. 0. 0. 0.]
[1. 1. 2. 0. 0.] [2. 2. 0. 0. 0.]
[1. 0. 2. 0. 0.] [2. 3. 0. 0. 0.]
Results are  [1. 3. 2. 0. 0.] [2. 0. 0. 0. 0.] -1.0
[3. 3. 2. 0. 0.] [0. 0. 0. 0. 0.]
[2. 2. 1. 0. 0.] [1. 1. 1. 0. 0.]
[1. 2. 1. 0. 0.] [2. 1. 1. 0. 0.]
[1. 2. 0. 0. 0.] [2. 1. 2. 0. 0.]
[1. 1. 0. 0. 0.] [2. 2. 2. 0. 0.]
Results are  [0. 0. 0. 0. 0.] [3. 3. 2. 0. 0.] 1.0
Results 1 -1.0


{'feature_idx': 1,
 'feature_value': -1.0,
 'groups': ((array([[ 0.5 ,  0.88,  1.  ],
          [-1.  ,  0.88,  0.5 ],
          [-1.  ,  0.5 ,  1.  ],
          [ 0.8 ,  0.88,  0.5 ],
          [ 1.  ,  0.5 ,  0.8 ],
          [ 0.88,  0.8 , -1.  ]]),
   array([0, 2, 1, 2, 1, 1]),
   array([1., 3., 2., 0., 0.])),
  (array([[-1., -1.,  1.],
          [ 1., -1., -1.]]),
   array([0, 0]),
   array([2., 0., 0., 0., 0.])))}

In [157]:
print(np.sum(data_samples[:, 254] <= 0.694))

9274


In [181]:
find_split_rf(data_samples, data_labels, 16, 10, 50, 'gini')

{'feature_idx': 208,
 'feature_value': 0.598,
 'groups': ((array([[-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          ...,
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          [-1.   , -1.   , -1.   , ..., -0.952, -1.   , -1.   ]]),
   array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2,
          5, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 3, 6, 2, 2, 2,
          2, 2, 2, 2, 2, 3, 2, 2, 4, 2]),
   array([ 0.,  0., 45.,  5.,  1.,  1.,  2.,  0.,  0.,  0.])),
  (array([[-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          [-1.   , -1.   , -1.   , ..., -0.671, -0.828, -1.   ],
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
          ...,
          [-1.   , -1.   , -1.   , ..., -1.   , -1.   , -1.   ],
     

In [217]:
root_node

{'feature_idx': 183,
 'feature_value': -1.0,
 'left': {'feature_idx': 34, 'feature_value': 1.0, 'left': 1, 'right': 1},
 'right': 3}

In [213]:
start = time.time()
root_node = build_tree(data_samples, data_labels, 10, 1, 16, 10, 'gini')
# find_split_rf(data_samples, data_labels, 16, 10, 'gini')
print(time.time() - start)

Depth is  1
Depth is  2
10.24358344078064


In [214]:
predictions = np.zeros(data_samples.shape[0])
for i in range(data_samples.shape[0]):
    print(f"Data {i}")
    predictions[i] = predict_node(data_samples[i,:], root_node)

Data 0
Data 1
Data 2
Data 3
Data 4
Data 5
Data 6
Data 7
Data 8
Data 9
Data 10
Data 11
Data 12
Data 13
Data 14
Data 15
Data 16
Data 17
Data 18
Data 19
Data 20
Data 21
Data 22
Data 23
Data 24
Data 25
Data 26
Data 27
Data 28
Data 29
Data 30
Data 31
Data 32
Data 33
Data 34
Data 35
Data 36
Data 37
Data 38
Data 39
Data 40
Data 41
Data 42
Data 43
Data 44
Data 45
Data 46
Data 47
Data 48
Data 49
Data 50
Data 51
Data 52
Data 53
Data 54
Data 55
Data 56
Data 57
Data 58
Data 59
Data 60
Data 61
Data 62
Data 63
Data 64
Data 65
Data 66
Data 67
Data 68
Data 69
Data 70
Data 71
Data 72
Data 73
Data 74
Data 75
Data 76
Data 77
Data 78
Data 79
Data 80
Data 81
Data 82
Data 83
Data 84
Data 85
Data 86
Data 87
Data 88
Data 89
Data 90
Data 91
Data 92
Data 93
Data 94
Data 95
Data 96
Data 97
Data 98
Data 99
Data 100
Data 101
Data 102
Data 103
Data 104
Data 105
Data 106
Data 107
Data 108
Data 109
Data 110
Data 111
Data 112
Data 113
Data 114
Data 115
Data 116
Data 117
Data 118
Data 119
Data 120
Data 121
Data 122
Dat

Data 4125
Data 4126
Data 4127
Data 4128
Data 4129
Data 4130
Data 4131
Data 4132
Data 4133
Data 4134
Data 4135
Data 4136
Data 4137
Data 4138
Data 4139
Data 4140
Data 4141
Data 4142
Data 4143
Data 4144
Data 4145
Data 4146
Data 4147
Data 4148
Data 4149
Data 4150
Data 4151
Data 4152
Data 4153
Data 4154
Data 4155
Data 4156
Data 4157
Data 4158
Data 4159
Data 4160
Data 4161
Data 4162
Data 4163
Data 4164
Data 4165
Data 4166
Data 4167
Data 4168
Data 4169
Data 4170
Data 4171
Data 4172
Data 4173
Data 4174
Data 4175
Data 4176
Data 4177
Data 4178
Data 4179
Data 4180
Data 4181
Data 4182
Data 4183
Data 4184
Data 4185
Data 4186
Data 4187
Data 4188
Data 4189
Data 4190
Data 4191
Data 4192
Data 4193
Data 4194
Data 4195
Data 4196
Data 4197
Data 4198
Data 4199
Data 4200
Data 4201
Data 4202
Data 4203
Data 4204
Data 4205
Data 4206
Data 4207
Data 4208
Data 4209
Data 4210
Data 4211
Data 4212
Data 4213
Data 4214
Data 4215
Data 4216
Data 4217
Data 4218
Data 4219
Data 4220
Data 4221
Data 4222
Data 4223
Data 4224


Data 8124
Data 8125
Data 8126
Data 8127
Data 8128
Data 8129
Data 8130
Data 8131
Data 8132
Data 8133
Data 8134
Data 8135
Data 8136
Data 8137
Data 8138
Data 8139
Data 8140
Data 8141
Data 8142
Data 8143
Data 8144
Data 8145
Data 8146
Data 8147
Data 8148
Data 8149
Data 8150
Data 8151
Data 8152
Data 8153
Data 8154
Data 8155
Data 8156
Data 8157
Data 8158
Data 8159
Data 8160
Data 8161
Data 8162
Data 8163
Data 8164
Data 8165
Data 8166
Data 8167
Data 8168
Data 8169
Data 8170
Data 8171
Data 8172
Data 8173
Data 8174
Data 8175
Data 8176
Data 8177
Data 8178
Data 8179
Data 8180
Data 8181
Data 8182
Data 8183
Data 8184
Data 8185
Data 8186
Data 8187
Data 8188
Data 8189
Data 8190
Data 8191
Data 8192
Data 8193
Data 8194
Data 8195
Data 8196
Data 8197
Data 8198
Data 8199
Data 8200
Data 8201
Data 8202
Data 8203
Data 8204
Data 8205
Data 8206
Data 8207
Data 8208
Data 8209
Data 8210
Data 8211
Data 8212
Data 8213
Data 8214
Data 8215
Data 8216
Data 8217
Data 8218
Data 8219
Data 8220
Data 8221
Data 8222
Data 8223


In [215]:
print(np.sum(predictions != 0))

9298


In [216]:
print(np.mean(predictions == data_labels))

0.21703592170359218


In [147]:
root_node

{'feature_idx': 223,
 'feature_value': 0.72,
 'left': 2,
 'right': {'feature_idx': 254,
  'feature_value': 0.694,
  'left': 2,
  'right': {'feature_idx': 15,
   'feature_value': 0.51,
   'left': 4,
   'right': {'feature_idx': 192,
    'feature_value': 0.97,
    'left': 2,
    'right': {'feature_idx': 241,
     'feature_value': 0.851,
     'left': 2,
     'right': {'feature_idx': 207,
      'feature_value': 0.884,
      'left': 2,
      'right': {'feature_idx': 241,
       'feature_value': 0.803,
       'left': 3,
       'right': {'feature_idx': 224,
        'feature_value': 0.828,
        'left': 2,
        'right': {'feature_idx': 47,
         'feature_value': 0.956,
         'left': 4,
         'right': {'feature_idx': 255,
          'feature_value': -0.386,
          'left': 2,
          'right': {'feature_idx': 223,
           'feature_value': 0.718,
           'left': 3,
           'right': {'feature_idx': 47,
            'feature_value': 0.937,
            'left': 5,
            

In [132]:
np.sum(data_samples[:, 239] <= 0.661)

9286

In [133]:
data_samples.shape[0]

9298

In [119]:
np.sum(data_samples[:, 175] <= 0.968) - data_samples.shape[0]

-1

In [102]:
print(toy_data)
print(result_dict['groups'][0])
print(result_dict['groups'][1])
print(terminate_node(result_dict['groups'][0]))

[[-1.   -1.    1.  ]
 [ 0.5   0.88  1.  ]
 [-1.    0.88  0.5 ]
 [-1.    0.5   1.  ]
 [ 0.8   0.88  0.5 ]
 [ 1.    0.5   0.8 ]
 [ 1.   -1.   -1.  ]
 [ 0.88  0.8  -1.  ]]
(array([[ 0.5 ,  0.88,  1.  ],
       [-1.  ,  0.88,  0.5 ],
       [-1.  ,  0.5 ,  1.  ],
       [ 0.8 ,  0.88,  0.5 ],
       [ 1.  ,  0.5 ,  0.8 ],
       [ 0.88,  0.8 , -1.  ]]), array([0, 2, 1, 2, 1, 1]), array([1., 3., 2., 0., 0.]))
(array([[-1., -1.,  1.],
       [ 1., -1., -1.]]), array([0, 0]), array([2., 0., 0., 0., 0.]))
1


In [106]:
isPure(result_dict['groups'][0])

False

In [79]:
tree = sc.DecisionTreeClassifier(max_depth = 1, random_state = 0)
tree.fit(toy_data, labels)
tree.predict(toy_data)

array([0, 1, 1, 1, 1, 1, 0, 1])

In [63]:
# Testing space
feature_vect = np.array([0.1, 0.5, 0.05, 0.1, 8, 8, 8, 9])
labels = np.array([0, 0, 2, 1, 2, 1, 0, 1])
best_val, split_val, cc_left_final, cc_right_final = best_split_along_feature(feature_vect, labels, 5, split_function = rel_entropy)

print(best_val, split_val)
print(cc_left_final, cc_right_final)

1.4488156357251847 0.05
[3. 3. 1. 0. 0.] [0. 0. 1. 0. 0.]


In [24]:
sort_idx = np.argsort(feature_vect)
sorted_labels = labels[sort_idx]
sorted_fv = feature_vect[sort_idx]

n_points = labels.shape[0]
    
# Left counts: Initially, everything
_, cc_left = np.unique(labels, return_counts = True)
tot_count_left = n_points

# Right counts: Initially nothing
cc_right = np.zeros(cc_left.shape)
tot_count_right = 0

# Testing
split = 1

print("Start")
print(cc_left, cc_right)
start_idx = 0; end_idx = 1
while start_idx < n_points:
    print(f"Split {split}")
    while end_idx < n_points and (sorted_fv[end_idx] == sorted_fv[end_idx - 1]):
        end_idx += 1
        
    chunk = sorted_labels[start_idx : end_idx]
    chunk_labels, counts = np.unique(chunk, return_counts = True)
    
    split += 1
    
    # Move counts to the right portion
    cc_left[chunk_labels] -= counts
    cc_right[chunk_labels] += counts
    
    tot_count_left -= np.sum(counts)
    tot_count_right += np.sum(counts)
    
    start_idx = end_idx; end_idx += 1
    
    print(cc_left, cc_right)
    print(tot_count_left, tot_count_right)

Start
[3 3 2] [0. 0. 0.]
Split 1
[3 3 1] [0. 0. 1.]
7 1
Split 2
[2 2 1] [1. 1. 1.]
5 3
Split 3
[1 2 1] [2. 1. 1.]
4 4
Split 4
[0 1 0] [3. 2. 2.]
1 7
Split 5
[0 0 0] [3. 3. 2.]
0 8


In [35]:
print(sorted_labels)
print(sorted_fv)

[2 0 1 0 2 1 0 1]
[0.05 0.1  0.1  0.5  8.   8.   8.   9.  ]


In [96]:
def alterdict(dic):
    del(dic['old'])
    dic['new'] = 88
    

a = {'old' : 83}

print(a)
alterdict(a)
print(a)

{'old': 83}
{'new': 88}
